Wahlpflichtfach Künstliche Intelligenz II: Praktikum 

---

# 01 - Natural Language Processing (NLP)

Im Folgenden wird ein [CoNLL-2003](https://huggingface.co/datasets/imdb) Datensatz verwendet.

In [1]:
from datasets import load_dataset

dataset = load_dataset("conll2003")

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [3]:
dataset["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [4]:
label_list = dataset["train"].features[f"ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

## Tokenizer und Model

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-cased')

In [6]:
def tokenize_and_align_labels(ds):
    tokenized_inputs = tokenizer(ds["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(ds["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx: 
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [7]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

In [8]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

2024-02-21 21:24:30.751643: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-21 21:24:30.772813: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 21:24:30.772841: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 21:24:30.773575: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-21 21:24:30.778352: I tensorflow/core/platform/cpu_feature_guar

In [9]:
id2label = {i: name for i, name in enumerate(label_list)}
label2id = {name: i for i, name in enumerate(label_list)}

### Modeling

In [10]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(label_list), id2label=id2label, label2id=label2id)

2024-02-21 21:24:32.213118: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-21 21:24:32.237089: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-21 21:24:32.237131: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-21 21:24:32.239594: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-21 21:24:32.239631: I external/local_xla/xla/stream_executor

In [59]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_dataset["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_dataset["validation"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_dataset["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [16]:
import numpy as np
from seqeval.metrics import f1_score, precision_score, recall_score


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return {
        "precision": precision_score(true_labels, true_predictions),
        "recall": recall_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
    }

In [17]:
from tensorflow.keras.optimizers import Adam

from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

model.compile(optimizer=Adam(learning_rate=5e-5))
model.fit(tf_train_set, validation_data=tf_validation_set, epochs=1, callbacks=[metric_callback])

877/877 [==============================] - 76s 76ms/step - loss: 0.0198 - val_loss: 0.0569 - precision: 0.9111 - recall: 0.9280 - f1: 0.9195


# Evaluate

In [21]:
inputs = tokenizer(dataset["test"][0]["tokens"], return_tensors="tf", truncation=True, is_split_into_words=True)
logits = model(**inputs).logits
inputs

In [27]:
from tensorflow.math import argmax

predicted_token_class_ids = argmax(logits, axis=-1)


predicted_token_class = [model.config.id2label[t] for t in predicted_token_class_ids[0].numpy().tolist()]
predicted_token_class

33

In [36]:
dataset["test"][0]["tokens"]

['SOCCER',
 '-',
 'JAPAN',
 'GET',
 'LUCKY',
 'WIN',
 ',',
 'CHINA',
 'IN',
 'SURPRISE',
 'DEFEAT',
 '.']

In [61]:
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
sep = [i for i, t in enumerate(tokens) if "##" in t]

" ".join(tokens).replace(" ##", "")

'[CLS] SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT . [SEP]'

In [62]:
token_class = ""
for i, p in enumerate(predicted_token_class):
    if i + 1 in sep:
        token_class += p + ","
        continue
    token_class += p + " "

token_class

'O O,O,O,O O B-PER,O,I-ORG O,O O,I-ORG,O,O O,O O B-PER,I-ORG,O O B-LOC,O,I-MISC,I-MISC,I-MISC O,O,O,O O O '

---

Wahlpflichtach Künstliche Intelligenz II: Praktikum 